# Overwatch League Modified Datasets Details
## Author: Christian Smith

These are the all the details and proceedures for how I made the modified datasets that I used in the "The Overwatch League: Statistics For Success" article.

## Necessary Python Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## stats2022 Dataframe

This is the process for how I made the "stats2022" dataframe used as the primary dataframe in both the EDA and modeling sections:  
Note: original datasets are "2022 Player Stats" and "Map Stats" found here: [Official Overwatch League Statistics](https://overwatchleague.com/en-us/statslab)

In [2]:
# 1) Start by loading in the 2022 match data provided by the Overwatch League
owl_2022 = pd.read_csv('phs-2022.csv')

# 2) Create a new dataframe that is a subset of this dataframe containing only the eight important stats mentioned
#    in the article, and only containing the sets of data where the player was playing as "All Heroes"
stats = ['All Damage Done', 'Eliminations', 'Healing Done', 
         'Average Time Alive', 'Time Building Ultimate', 'Ultimates Used', 'Deaths', 'Damage Blocked']
less_2022 = pd.DataFrame(owl_2022.loc[(owl_2022['stat_name'].isin(stats)) & (owl_2022['hero_name'] == 'All Heroes')])

# 3) Create new dataframes for each statistic that is going to be analyzed
all_dmg = pd.DataFrame(less_2022.loc[(less_2022['stat_name'] == 'All Damage Done') & (less_2022['hero_name'] == 'All Heroes')])
all_dmg.columns = all_dmg.columns.str.replace('esports_match_id', 'match_id')

elims = pd.DataFrame(less_2022.loc[(less_2022['stat_name'] == 'Eliminations') & (less_2022['hero_name'] == 'All Heroes')])
elims.columns = elims.columns.str.replace('esports_match_id', 'match_id')

heals = pd.DataFrame(less_2022.loc[(less_2022['stat_name'] == 'Healing Done') & (less_2022['hero_name'] == 'All Heroes')])
heals.columns = heals.columns.str.replace('esports_match_id', 'match_id')

talive = pd.DataFrame(less_2022.loc[(less_2022['stat_name'] == 'Average Time Alive') & (less_2022['hero_name'] == 'All Heroes')])
talive.columns = talive.columns.str.replace('esports_match_id', 'match_id')

ultbuild = pd.DataFrame(less_2022.loc[(less_2022['stat_name'] == 'Time Building Ultimate') & (less_2022['hero_name'] == 'All Heroes')])
ultbuild.columns = ultbuild.columns.str.replace('esports_match_id', 'match_id')

ultsused = pd.DataFrame(less_2022.loc[(less_2022['stat_name'] == 'Ultimates Used') & (less_2022['hero_name'] == 'All Heroes')])
ultsused.columns = ultsused.columns.str.replace('esports_match_id', 'match_id')

deaths = pd.DataFrame(less_2022.loc[(less_2022['stat_name'] == 'Deaths') & (less_2022['hero_name'] == 'All Heroes')])
deaths.columns = deaths.columns.str.replace('esports_match_id', 'match_id')

dmgblock = pd.DataFrame(less_2022.loc[(less_2022['stat_name'] == 'Damage Blocked') & (less_2022['hero_name'] == 'All Heroes')])
dmgblock.columns = dmgblock.columns.str.replace('esports_match_id', 'match_id')

# 4) In order to add whether a player won or not, we will need match data from the map the player played in.
#    To do this, we will use the "Map Stats" dataframe
match_data = pd.read_csv('match_map_stats.csv')

# 5) We will then subset the "Map Stats" dataframe by only the events that happened in 2022 and reduce them to one row per map
res_2022 = pd.DataFrame(match_data[match_data['stage'].str.contains('2022')])
single_match = res_2022.drop_duplicates(subset=["match_id", "map_name"], keep='first')
single_match = single_match.drop(['attacker_payload_distance', 'defender_payload_distance',
                                  'attacker_time_banked', 'defender_time_banked',
                                  'attacker_control_perecent', 'defender_control_perecent',
                                  'attacker_round_end_score', 'defender_round_end_score','map_round', 
                                  'winning_team_final_map_score',
                                  'losing_team_final_map_score', 'control_round_name', 'Attacker',
                                  'Defender', 'team_one_name', 'team_two_name'], axis = 1)

# 6) We will then merge each of the modified stats datasets with the "single_match" dataframe to attach player's stats to each map
dmgwin = pd.merge(all_dmg, single_match, on = ['match_id', 'map_name'])

elimswin = pd.merge(elims, single_match, on = ['match_id', 'map_name'])

healswin = pd.merge(heals, single_match, on = ['match_id', 'map_name'])

talivewin = pd.merge(talive, single_match, on = ['match_id', 'map_name'])

ultbuildwin = pd.merge(ultbuild, single_match, on = ['match_id', 'map_name'])

ultsusedwin = pd.merge(ultsused, single_match, on = ['match_id', 'map_name'])

deathswin = pd.merge(deaths, single_match, on = ['match_id', 'map_name'])

dmgblockwin = pd.merge(dmgblock, single_match, on = ['match_id', 'map_name'])

#7) We will then apply the "Winner" column to each of the dataframes using the "is_winner()" function
def is_winner(df):
    if (df['map_winner'] == df['team_name']):
        winner = 1.0
    else:
        winner = 0.0
    return winner

dmgwin['winner'] = dmgwin.apply(is_winner, axis = 1)

elimswin['winner'] = elimswin.apply(is_winner, axis = 1)

healswin['winner'] = healswin.apply(is_winner, axis = 1)

talivewin['winner'] = talivewin.apply(is_winner, axis = 1)

ultbuildwin['winner'] = ultbuildwin.apply(is_winner, axis = 1)

ultsusedwin['winner'] = ultsusedwin.apply(is_winner, axis = 1)

deathswin['winner'] = deathswin.apply(is_winner, axis = 1)

dmgblockwin['winner'] = dmgblockwin.apply(is_winner, axis = 1)

# 8) We will now drop all unecessary columns of each dataframe
def drop_cols(df):
    df.drop(['start_time', 'tournament_title', 'map_type',
             'hero_name', 'round_start_time', 'round_end_time', 'stage', 'game_number',
             'match_winner', 'map_winner', 'map_loser'], axis = 1, inplace = True)

drop_cols(dmgwin)
drop_cols(healswin)
drop_cols(elimswin)
drop_cols(dmgblockwin)
drop_cols(deathswin)
drop_cols(ultsusedwin)
drop_cols(ultbuildwin)
drop_cols(talivewin)

# 9) We will now merge all of the dataframes to transfer the row based stats of the original dataframe to column based ones
def mergedfs(largedf, smalldf, smalldf2_name, short_name):
    merged_df = pd.merge(largedf, smalldf, 
                    how='outer', 
                    on=['match_id', 'map_name', 'player_name'])
    merged_df[['stat_name_y', 'amount_y']] = merged_df[['stat_name_y', 'amount_y']].fillna(value= {'stat_name_y': smalldf2_name, 'amount_y': 0})
    merged_df.rename(columns = {'stat_name_y':smalldf2_name, 'amount_y':short_name, 'winner_x':'winner',
                                'team_name_x':'team_name', 'stat_name_x':'stat_name', 'amount_x':'amount'}, inplace = True)
    merged_df.drop(['winner_y', 'team_name_y'], axis = 1, inplace = True)
    return merged_df

dmgheals = mergedfs(dmgwin, healswin, 'Healing Done', 'Heals')[:-2]

dmgelims = mergedfs(dmgheals, elimswin, 'Eliminations', 'Elims')

dmgdeaths = mergedfs(dmgelims, deathswin, 'Total Deaths', 'Deaths')[:-2]

dmgblocks = mergedfs(dmgdeaths, dmgblockwin, 'Total Damage Blocked', 'Damage Blocked')

dmgults = mergedfs(dmgblocks, ultsusedwin, 'Total Ults Used', 'Ultimates Used')[:-2]

dmgbuildult = mergedfs(dmgults, ultbuildwin, 'Total Time Building Ult', 'Average Time Building Ultimate')[:-2]

all_stats = mergedfs(dmgbuildult, talivewin, 'Avg Time Alive', 'Average Time Alive')[:-2]

all_stats = all_stats.drop_duplicates(['match_id', 'map_name', 'player_name'])

# 10) We will now rename certain columns to better clarify what the values contain and remove redundancy
all_stats = all_stats.drop(['stat_name', 'Healing Done', 'Eliminations', 'Total Deaths', 'Total Damage Blocked',
                            'Total Ults Used', 'Total Time Building Ult', 'Avg Time Alive'], axis = 1)
all_stats = all_stats.rename(columns = {'amount':'Damage', 'Heals':'Healing', 'team_name':'Team', 'player_name':'Player',
                                        'map_name':'Map', 'match_id':'Match ID', 'Elims':'Eliminations', 'winner':'Winner'})

# 11) Now, we will move onto adding the "Role" column. First, we will find all the hero-specific columns
hero_stats = owl_2022[owl_2022['hero_name'] != 'All Heroes']

# 12) Next, because each player must play one role in each map, we may only keep the first hero they played in a map to determine their role
hero_single = hero_stats[hero_stats['stat_name'] == 'All Damage Done'].drop_duplicates(subset=['esports_match_id', 'player_name', 'map_name'], keep='first')

# 13) Then, we will add the "Role" column to the hero-based dataframe
def role_type(df):
    role = 0
    
    supports = ['Ana', 'Baptiste', 'Brigitte', 'Kiriko', 'Lucio', 'Mercy', 'Moira', 'Zenyatta']
    tanks = ['D.Va', 'Doomfist', 'Junker Queen', 'Orisa', 'Reinhardt', 'Roadhog', 'Sigma', 'Winston', 'Wrecking Ball', 'Zarya']
    
    if (df['hero_name'] in tanks):
        role = 1
    elif (df['hero_name'] in supports):
        role = 3
    else:
        role = 2
    return role

hero_single['Role'] = hero_single.apply(role_type, axis = 1)

# 14) We will now reduce the hero-based dataframe to only the required identifiers to merge and the role of the player in that map 
hero_role = hero_single.drop(['start_time', 'tournament_title', 'map_type', 'stat_name', 'team_name', 'hero_name', 'amount'], axis = 1)
hero_role = hero_role.rename(columns = {'esports_match_id':'Match ID', 'map_name':'Map', 'player_name':'Player'})

# 15) We will merge the hero-based dataframe now only containing role information with our primary dataframe
stats2022 = pd.merge(all_stats, hero_role, how = 'outer').dropna()
stats2022 = stats2022[['Match ID', 'Map', 'Player', 'Team', 'Damage', 'Healing',
       'Eliminations', 'Deaths', 'Damage Blocked', 'Ultimates Used',
       'Average Time Building Ultimate', 'Average Time Alive', 'Role', 'Winner']]

In [3]:
stats2022.head()

,Match ID,Map,Player,Team,Damage,Healing,Eliminations,Deaths,Damage Blocked,Ultimates Used,Average Time Building Ultimate,Average Time Alive,Role,Winner
0,38971,Lijiang Tower,FunnyAstro,Los Angeles Gladiators,11560.871071,10333.540760,20.0,6.0,0.000000,4.0,731.976021,83.394003,3,1.0
1,38971,Lijiang Tower,Patiphan,Los Angeles Gladiators,12987.519505,0.000000,27.0,12.0,0.000000,4.0,695.848020,39.655501,2,1.0
2,38971,Lijiang Tower,Myunb0ng,New York Excelsior,5809.344608,13481.569533,18.0,12.0,0.000000,6.0,698.306020,45.837735,3,0.0
3,38971,Lijiang Tower,Yaki,New York Excelsior,16779.776752,61.864814,28.0,12.0,567.380009,5.0,764.867022,57.447002,2,0.0
4,38971,Lijiang Tower,Reiner,Los Angeles Gladiators,12185.249345,0.000000,24.0,9.0,19006.432701,4.0,637.724018,43.295229,1,1.0


## game_one Dataframe

This is the process for how I made the "game_one" dataframe used to visualize how often the first map winner went on to win the entire match in the EDA section:  
Note: original dataset is the "Map Stats" found here: [Official Overwatch League Statistics](https://overwatchleague.com/en-us/statslab)

In [12]:
# 1) We will start by using the "single_match" dataframe created above when constructing the "stats2022" dataframe by
#    creating a new dataframe based on only the first maps of each match played
game_one = single_match[single_match['game_number'] == 1].copy()

# 2) We will then define a function that will apply whether or not the first map winner went on to win the entire match
def first_victor(df):
    if (df['map_winner'] == df['match_winner']):
        first_victor = 1
    else:
        first_victor = 0
    return first_victor

# 3) We will apply the first_victor column to the dataframe
game_one['first_victor'] = game_one.apply(first_victor,  axis = 1)

In [14]:
game_one.head()

,round_start_time,round_end_time,stage,match_id,game_number,match_winner,map_winner,map_loser,map_name,first_victor
11225,05/05/22 19:10,05/05/22 19:16,2022: Kickoff Clash: Qualifiers,38971,1,Los Angeles Gladiators,Los Angeles Gladiators,New York Excelsior,Lijiang Tower,1
11233,05/05/22 21:01,05/05/22 21:05,2022: Kickoff Clash: Qualifiers,38972,1,San Francisco Shock,San Francisco Shock,Vegas Eternal,Ilios,1
11239,05/05/22 22:23,05/05/22 22:28,2022: Kickoff Clash: Qualifiers,38973,1,Atlanta Reign,Florida Mayhem,Atlanta Reign,Lijiang Tower,0
11249,05/06/22 19:06,05/06/22 19:10,2022: Kickoff Clash: Qualifiers,38975,1,San Francisco Shock,San Francisco Shock,London Spitfire,Lijiang Tower,1
11255,05/06/22 20:25,05/06/22 20:31,2022: Kickoff Clash: Qualifiers,38974,1,Boston Uprising,Boston Uprising,Vancouver Titans,Ilios,1
